In [2]:
# Set up for running the code
import pandas
import math
import numpy
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from yellowbrick.classifier import ClassificationReport,ConfusionMatrix


# Read in the data from the CSV file
dataframe = pandas.read_csv("Crime_Data_from_2020_to_Present.csv", dtype={'TIME OCC': str})


In [17]:
# Get only the date in the DATE OCC column
dataframe['DATE OCC'] = dataframe['DATE OCC'].str.split().str.get(0)

In [18]:
# Trim down the data to only have agregous crimes
nonAgregousCrimeCDs = [
    648,
    220,
    840,
    948,
    485,
    480,
    432,
    487,
    755,
    761,
    942,
    666,
    662,
    664,
    310,
    330,
    410,
    320,
    870,
    235,
    627,
    813,
    237,
    814,
    922,
    944,
    903,
    954,
    660,
    654,
    653,
    930,
    812,
    943,
    951,
    950,
    345,
    444,
    445,
    880,
    886,
    649,
    652,
    651,
    433,
    865,
    353,
    668,
    670,
    940,
    884,
    890,
    434,
    439,
    904,
    906,
    349,
    347,
    949,
    830,
    850,
    236,
    626,
    956,
    762,
    760,
    435,
    436,
    820,
    946,
    806,
    932,
    446,
    352,
    452,
    805,
    933,
    351,
    451,
    438,
    931,
    437,
    210,
    845,
    810,
    815,
    443,
    442,
    343,
    821,
    763,
    924,
    421,
    331,
    420,
    450,
    354,
    441,
    440,
    475,
    473,
    474,
    350,
    341,
    928,
    647,
    470,
    471,
    888,
    661,
    740,
    745,
    520,
    510,
    522,
    900,
    901,
    902,
    756
]

for cd in nonAgregousCrimeCDs:
    dataframe = dataframe[dataframe["Crm Cd 1"] != cd]

dataframe = dataframe.dropna(subset=['Crm Cd 1'])

In [4]:
print(pandas.unique(dataframe['Crm Cd 1']))


# print(pandas.unique(dataframe['AREA']))

[624. 845. 745. 740. 121. 442. 946. 341. 330. 930. 648. 626. 440. 354.
 210. 230. 310. 510. 420. 761. 236. 662. 350. 860. 480. 623. 956. 900.
 888. 331. 901. 886. 421. 647. 940. 810. 922. 812. 220. 625. 755. 649.
 434. 815. 251. 320. 850. 668. 902. 664. 920. 343. 437. 753. 928. 910.
 760. 762. 661. 351. 821. 237. 903. 813. 666. 820. 627. 805. 763. 441.
 122. 443. 450. 410. 890. 520. 670. 951. 660. 654. 352. 250. 110. 933.
 652. 950. 231. 345. 822. 814. 932. 622. 471. 235. 470. 921. 906. 433.
 651. 943. 653. 436. 439. 806. 949. 522. 446. 113. 438. 451. 521. 485.
 944. 954. 756. 942. 487.  nan 473. 347. 435. 880. 444. 475. 474. 931.
 865. 349. 430. 452. 353. 870. 924. 840. 948. 884. 904. 830. 432. 882.
 445.]
